In [1]:
import numpy as np
import sensors
import ecm_determ

In [2]:
DIST_RATIO = 0.5

In [3]:
Num_sensors1 = 5
Num_emitters1 = 1
sample_size1 = 100
theta1_rad = [0.5] # Направление прибытия (DOA) в радианах
theta1_deg = np.rad2deg(theta1_rad) # Направление прибытия (DOA) в градусах
#Q1 = 1.1 * np.eye(Num_sensors1, dtype=np.float64) # Ковариация шума
Q1 = sensors.random_complex_cov(n=Num_sensors1, max_real=1.2, seed=50)
A1 = np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors1).reshape(-1,1) * np.sin(theta1_rad)) # Матрица управляющих векторов
# Генерация векторов сигнала, шума и принятого сигнала
S1 = sensors.gds(Num_emitters1, sample_size1)
n1 = sensors.gss(Num_sensors1, sample_size1, Q1)
X1 = (A1 @ S1.T + n1.T).T

In [4]:
X1_with_mv = sensors.MCAR(X1, [2,4], [40, 40])

In [5]:
EM_theta1_rad, EM_S1, EM_Q1, lhd_1 = ecm_determ.multi_start_ECM_un(X1_with_mv, Num_emitters1, num_of_starts=10, max_iter=10, rtol=1e-6)

0-th start
Initial theta = [-2.83488836]
incomplete likelihood is -200.224454753177 on iteration 0
incomplete likelihood is -108.148787381567 on iteration 1
incomplete likelihood is -129.99410955188955 on iteration 2
incomplete likelihood is -141.70048569926422 on iteration 3
incomplete likelihood is -148.6617344444916 on iteration 4
incomplete likelihood is -149.95248447212845 on iteration 5
incomplete likelihood is -143.33288017187513 on iteration 6
incomplete likelihood is -125.38382351277711 on iteration 7
incomplete likelihood is -89.34751462499385 on iteration 8
incomplete likelihood is -20.645136837392755 on iteration 9
1-th start
Initial theta = [-2.86887136]
incomplete likelihood is -444.91206211629844 on iteration 0
incomplete likelihood is -390.8984296318142 on iteration 1
incomplete likelihood is -343.44200393134787 on iteration 2
incomplete likelihood is -354.5726690424309 on iteration 3
incomplete likelihood is -375.02419472819196 on iteration 4
incomplete likelihood is -

In [6]:
EM_theta1_deg = np.rad2deg(EM_theta1_rad)
print(f"Погрешность в смысле разности углов в градусах: {np.abs(EM_theta1_deg-theta1_deg)},")
print(f"Погрешность в смысле разности синусов углов: {np.abs(np.sin(EM_theta1_rad)-np.sin(theta1_rad))}.")

Погрешность в смысле разности углов в градусах: [0.01175359],
Погрешность в смысле разности синусов углов: [0.00018004].


In [7]:
EM_Q1-Q1

array([[ 0.04495054+0.00000000e+00j, -0.03992931-1.06666705e-01j,
        -0.11062603-1.45199213e-02j, -0.03914684-4.76145836e-02j,
         0.1048952 -5.52034130e-02j],
       [-0.03992931+1.06666705e-01j, -0.19688184+0.00000000e+00j,
        -0.01307358-5.20634956e-02j,  0.06834897+9.67301187e-03j,
        -0.04548263+3.58096838e-02j],
       [-0.11062603+1.45199213e-02j, -0.01307358+5.20634956e-02j,
         0.54770267+0.00000000e+00j,  0.09709726+1.09144010e-01j,
        -0.25519261+1.62557005e-01j],
       [-0.03914684+4.76145836e-02j,  0.06834897-9.67301187e-03j,
         0.09709726-1.09144010e-01j,  0.17999484+0.00000000e+00j,
         0.01797322+1.38056074e-01j],
       [ 0.1048952 +5.52034130e-02j, -0.04548263-3.58096838e-02j,
        -0.25519261-1.62557005e-01j,  0.01797322-1.38056074e-01j,
         1.03746828+1.74652275e-19j]])

In [8]:
print(np.round(Q1-EM_Q1, 3))

[[-0.045+0.j     0.04 +0.107j  0.111+0.015j  0.039+0.048j -0.105+0.055j]
 [ 0.04 -0.107j  0.197+0.j     0.013+0.052j -0.068-0.01j   0.045-0.036j]
 [ 0.111-0.015j  0.013-0.052j -0.548+0.j    -0.097-0.109j  0.255-0.163j]
 [ 0.039-0.048j -0.068+0.01j  -0.097+0.109j -0.18 +0.j    -0.018-0.138j]
 [-0.105-0.055j  0.045+0.036j  0.255+0.163j -0.018+0.138j -1.037-0.j   ]]


In [9]:
for row in EM_Q1 - Q1:
    print(" ".join(f"{x:.3f}" for x in row))

0.045+0.000j -0.040-0.107j -0.111-0.015j -0.039-0.048j 0.105-0.055j
-0.040+0.107j -0.197+0.000j -0.013-0.052j 0.068+0.010j -0.045+0.036j
-0.111+0.015j -0.013+0.052j 0.548+0.000j 0.097+0.109j -0.255+0.163j
-0.039+0.048j 0.068-0.010j 0.097-0.109j 0.180+0.000j 0.018+0.138j
0.105+0.055j -0.045-0.036j -0.255-0.163j 0.018-0.138j 1.037+0.000j


In [10]:
for row in Q1:
    print(" ".join(f"{x:.3f}" for x in row))

0.225+0.000j -0.012-0.041j -0.089+0.034j -0.060-0.015j 0.060+0.009j
-0.012+0.041j 0.290+0.000j 0.036-0.057j -0.041-0.029j 0.042+0.069j
-0.089-0.034j 0.036+0.057j 0.117+0.000j 0.097-0.062j -0.087+0.007j
-0.060+0.015j -0.041+0.029j 0.097+0.062j 0.386+0.000j -0.083-0.152j
0.060-0.009j 0.042-0.069j -0.087-0.007j -0.083+0.152j 0.182-0.000j


In [11]:
for row in EM_Q1:
    print(" ".join(f"{x:.3f}" for x in row))

0.270+0.000j -0.052-0.148j -0.200+0.019j -0.099-0.062j 0.165-0.047j
-0.052+0.148j 0.093+0.000j 0.023-0.109j 0.028-0.019j -0.003+0.104j
-0.200-0.019j 0.023+0.109j 0.664+0.000j 0.194+0.047j -0.342+0.170j
-0.099+0.062j 0.028+0.019j 0.194-0.047j 0.566+0.000j -0.065-0.014j
0.165+0.047j -0.003-0.104j -0.342-0.170j -0.065+0.014j 1.219+0.000j


In [12]:
EM_theta1_deg

array([28.63613617])

In [13]:
theta1_deg

array([28.64788976])